In [54]:
# Setup: python3
# %pip install ipykernel
# %pip install pandas
# %pip install numpy
# %pip install matplotlib
# %pip install openpyxl
# %pip install xlrd
# %pip install pulp

In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pulp import *
import openpyxl
import xlrd

# Read the data from the excel file data\punto 1.xlsx
Table_1 = pd.read_excel('data\punto 1.xlsx', sheet_name='Table_1')
Table_2 = pd.read_excel('data\punto 1.xlsx', sheet_name='Table_2')
# Print the data
display(Table_1, Table_2)

,Producto,Demanda Mensual (cajas),Cp,Cm ($/(caja*mes)),Cajas/tarima,Cajas/viaje Despacho
0,A,3000,100,40,6,3.0
1,B,4000,100,45,2,1.5
2,C,3500,100,35,4,2.5


,Sección,Distancia a Recepción (m),Distancia a Despacho (m)
0,1,137,112
1,2,121,96
2,3,105,80
3,4,106,48
4,5,121,32
5,6,107,110
6,7,91,94
7,8,75,78
8,9,76,46
9,10,92,30


In [56]:
# for table 1, add column # Tarimas = Demanda Mensual (cajas)/Cajas/tarima
Table_1['# Tarimas'] = Table_1['Demanda Mensual (cajas)']/Table_1['Cajas/tarima']
# for table 1, add column Tarimas/viaje = 2
Table_1['Tarimas/viaje'] = 2
# for table 1, add column # Viajes Recepción = # Tarimas/Tarimas/viaje
Table_1['# Viajes Recepción'] = Table_1['# Tarimas']/Table_1['Tarimas/viaje']
# for table 1, add column # Viajes Despacho = Demanda Mensual (cajas)/Cajas/viaje Despacho
Table_1['# Viajes Despacho'] = Table_1['Demanda Mensual (cajas)']/Table_1['Cajas/viaje Despacho']
# Get Relation IN/OUT
Table_1['Relation IN/OUT'] = Table_1['# Viajes Recepción']/Table_1['# Viajes Despacho']
# for table 1, add column Q = square_root(2*Cp*Demanda Mensual (cajas) / Cm ($/(caja*mes)))
Table_1['Q'] = np.sqrt(2*Table_1['Cp']*Table_1['Demanda Mensual (cajas)']/Table_1['Cm ($/(caja*mes))'])
# for table 1. add column S = round up Q/Cajas/tarima
Table_1['S'] = np.ceil(Table_1['Q']/Table_1['Cajas/tarima'])
# for table 1, add column # Tarimas/Seccion = 10
Table_1['# Tarimas/Seccion'] = 10
# for table 1, add column # Secciones = S/# Tarimas/Seccion
Table_1['# Secciones'] =  np.ceil(+Table_1['S']/Table_1['# Tarimas/Seccion'])
# Ask how many sections are there
#Sections = int(input('How many sections are there? '))
Sections = 15
# if sum(# Secciones) <= Sections, tell, the number of sections is enough
if Table_1['# Secciones'].sum() <= Sections:
    print('The number of sections is enough')
# if sum(# Secciones) > Sections, tell, the number of sections is not enough
else:
    print('The number of sections is not enough')
# ask for the cost of Recepcion
#Cost_Recepcion = int(input('What is the cost of Recepcion? '))
Cost_Recepcion = 15
# ask for the cost per meter of Recepcion
#Cost_Recepcion_m = int(input('What is the cost per meter of Recepcion? '))
Cost_Recepcion_m = 2.0
# ask for the cost of Despacho
#Cost_Despacho = int(input('What is the cost of Despacho? '))
Cost_Despacho = 40
# ask for the cost per meter of Despacho
#Cost_Despacho_m = int(input('What is the cost per meter of Despacho? '))
Cost_Despacho_m = 0.5


The number of sections is enough


In [63]:
# Solution through PuLP
# indexes:
# i: products
# j: sections
# Variables:
# x[i,j]: 1 if product i is in section j, 0 otherwise
# Parameters:
# vr[i]: number of trips for receiving product i
# vd[i]: number of trips for delivering product i
# dr[j]: distance for receiving in section j 'Distancia a Recepción (m)' table 2
# dd[j]: distance for delivering in section j 'Distancia a Despacho (m)' table 2
# cr_m: cost per meter for receiving (travel must be done twice)
# cd_m: cost per meter for delivering (travel must be done twice)
# cr: cost per trip for receiving
# cd: cost per trip for delivering
# c[i,j]: cost of product i in section j
#        = 1/s[i]*(2*(cr_m*dr[j]*vr[i] + cd_m*dd[j]*vd[i])+cr*vr[i]+cd*vd[i])
# s[i]: number of sections needed for product i
# M: big number
# Objective:
# min sum(i,j) c[i,j]*x[i,j]
# Constraints:
# sum(j) x[i,j] = b[i] for all i
# sum(i) x[i,j] <= 1 for all j
# x[i,j] in {0,1} for all i,j

c = {}
for i in Table_1.index:
    for j in range(Sections):
        c[i,j] = 1/Table_1['# Secciones'][i]*(2*(Cost_Recepcion_m*Table_2['Distancia a Recepción (m)'][j]*Table_1['# Viajes Recepción'][i] + Cost_Despacho_m*Table_2['Distancia a Despacho (m)'][j]*Table_1['# Viajes Despacho'][i])+Cost_Recepcion*Table_1['# Viajes Recepción'][i]+Cost_Despacho*Table_1['# Viajes Despacho'][i])
# Create the 'prob' variable to contain the problem data
prob = LpProblem("The Warehouse Problem", LpMinimize)
# Create problem variables
x = LpVariable.dicts("x", (Table_1.index, range(Sections)), 0, 1, LpInteger)
# The objective function is added to 'prob' first
prob += lpSum([c[i,j]*x[i][j] for i in Table_1.index for j in range(Sections)])
# The constraints are added to 'prob'
for i in Table_1.index:
    prob += lpSum([x[i][j] for j in range(Sections)]) == Table_1['# Secciones'][i]
for j in range(Sections):
    prob += lpSum([x[i][j] for i in Table_1.index]) <= 1
# The problem data is written to an .lp file
prob.writeLP("Warehouse.lp")
# The problem is solved using PuLP's choice of Solver
prob.solve()
# The status of the solution is printed to the screen
print("Status:", LpStatus[prob.status])
# The objective value of the solution is printed to the screen
print("Total Cost = ", value(prob.objective))
# Each of the variables is printed with it's resolved optimum value
for v in prob.variables():
    if v.varValue > 0:
        print(v.name, "=", v.varValue)
# Print the solution as a table
sol = pd.DataFrame(index=range(Sections), columns=Table_1.index)
for i in Table_1.index:
    for j in range(Sections):
        sol[i][j] = x[i][j].varValue
display(sol)


Status: Optimal
Total Cost =  1185527.3809523808
x_0_1 = 1.0
x_0_4 = 1.0
x_0_5 = 1.0
x_1_11 = 1.0
x_1_12 = 1.0
x_1_13 = 1.0
x_1_14 = 1.0
x_1_7 = 1.0
x_1_8 = 1.0
x_1_9 = 1.0
x_2_10 = 1.0
x_2_2 = 1.0
x_2_3 = 1.0
x_2_6 = 1.0


c:\Users\Home\OneDrive\1_ Universidad\Logística II\Parcial_1-Python\.venv\Lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


,0,1,2
0,0.0,0.0,0.0
1,1.0,0.0,0.0
2,0.0,0.0,1.0
3,0.0,0.0,1.0
4,1.0,0.0,0.0
5,1.0,0.0,0.0
6,0.0,0.0,1.0
7,0.0,1.0,0.0
8,0.0,1.0,0.0
9,0.0,1.0,0.0
